In [ ]:
import pandas as pd
import numpy as np
#import bootstrapped.bootstrap as bts
#import bootstrapped.stats_functions as bs_stats
import os
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pylab as pl
from os import path
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"

In [ ]:
def P(t1,t2):
    p = (1/(1+t1))*1/t2*(np.exp(-t2)-np.exp(-2*t2))*(1/(1+2*t1))*1/(2*t2)*(np.exp(-2*t2)-np.exp(-4*t2))
    return p

In [ ]:
colors = ['aqua','slateblue','violet','deeppink','mediumorchid']
matplotlib.rcParams.update({'font.size': 22})       
fig, axs = plt.subplots(2,1,figsize=(8, 12))

proc_list = [250, 500, 1000]
sep = 125
for count,proc in enumerate(proc_list):
    v=0.5
    t_constrain = proc/(2*v)
    x = proc/sep
    L = 0.5*(10**(-0.08238 + 0.7258*np.log10(x) - 
                                 0.2514*(np.log10(x))**2 - 
                                 0.003995*(np.log10(x))**3 + 
                                 0.03445*(np.log10(x))**4 - 
                                 0.01077*(np.log10(x))**5 + 
                                 0.001371*(np.log10(x))**6 - 
                                 6.472*10**(-5)*(np.log10(x))**7))*sep
    t_extrusion = L/(4*v)
    t_loading  = proc/(v)*sep/L

    timescale1_o = t_loading/t_constrain
    timescale2_o = t_extrusion/t_constrain

    t_list = np.linspace(0.01,3.5)
    p1 = P(t_list,timescale2_o)
    p2 = P(timescale1_o,t_list)
    axs[0].plot(t_list,p1,'-',color=colors[count],markersize=15,linewidth=3)
    axs[0].scatter(timescale1_o ,P(timescale1_o,timescale2_o),s=100,c=colors[count],marker='o')



    axs[1].plot(t_list,p2,'-',color=colors[count],markersize=15,linewidth=3)
    axs[1].scatter(timescale2_o ,P(timescale1_o,timescale2_o),s=100,c=colors[count],marker='o')

axs[0].set_ylabel('P_ends-joined')
axs[0].set_xlabel('Tloading/Tconstrained')
legend = ['proc = ' +str(i)+' kb' for i in proc_list]

axs[0].legend(legend)
axs[0].tick_params(direction='out', length=8, width=2)
axs[0].set_ylim(0,0.5)

# Hide the right and top spines
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[0].yaxis.set_ticks_position('left')
axs[0].xaxis.set_ticks_position('bottom')

axs[1].set_ylabel('P_ends-joined')
axs[1].set_xlabel('Textrusion/Tconstrained')

axs[1].legend(legend)
axs[1].tick_params(direction='out', length=8, width=2)
axs[1].set_ylim(0,0.5)

# Hide the right and top spines
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[1].yaxis.set_ticks_position('left')
axs[1].xaxis.set_ticks_position('bottom')

plt.savefig('Figures/RelativeTimeScales.pdf',format='pdf',bbox_inches='tight')
plt.show()

In [ ]:
# relative timescale values calculated from Mathematica notebook "TimescaleHeatmap_20211004.nb"
loadconstrain = np.asarray([[2.41774, 1.54012, 1.08317, 0.829406, 0.678953],
                     [4.20978, 2.41774, 1.54012, 1.08317, 0.829406],
                     [8.03886, 4.20978, 2.41774, 1.54012, 1.08317],
                    [16.2482, 8.03886, 4.20978, 2.41774, 1.54012]])

extrusionconstrain = np.asarray([[0.413609, 0.324649, 0.230805, 0.15071, 0.0920534],
                     [0.475084, 0.413609, 0.324649, 0.230805, 0.15071],
                     [0.497583, 0.475084, 0.413609, 0.324649, 0.230805],
                    [0.492362, 0.497583, 0.475084, 0.413609, 0.324649]])
matplotlib.rcParams.update({'font.size': 22})       
fig, axs = plt.subplots(1,2,figsize=(18, 7))

dim1, dim2 = loadconstrain.shape
annot_loadconstrain = np.empty((dim1, dim2)).astype(str)
for i in range(dim1):
    for j in range(dim2):
        p = loadconstrain[i, j]
        annot_loadconstrain[i, j] = '%s' % (str(np.around(p,decimals=1)))  

dim1, dim2 = extrusionconstrain.shape
annot_extrusionconstrain = np.empty((dim1, dim2)).astype(str)
for i in range(dim1):
    for j in range(dim2):
        p = extrusionconstrain[i, j]
        annot_extrusionconstrain[i, j] = '%s' % (str(np.around(p,decimals=1)))  

proc_list_sorted = [62.5, 125, 250, 500, 1000]
proc_list_sorted = [str(p) for p in proc_list_sorted]

sep_list_sorted = [62.5, 125, 250, 500]
sep_list_sorted = [str(s) for s in sep_list_sorted]

sns.heatmap(pd.DataFrame(extrusionconstrain),annot=annot_extrusionconstrain,fmt='',xticklabels=proc_list_sorted,yticklabels=sep_list_sorted,cmap= 'viridis', vmin=0, vmax=17,cbar_kws={'label': 'ratio of loading time and constrain time'},ax=axs[1])
axs[1].set_ylabel('separation (kb)')
axs[1].set_xlabel('processivity (kb)')
axs[1].set_facecolor("0.8584083044982699")
axs[1].tick_params(direction='out', length=6, width=2)


sns.heatmap(pd.DataFrame(loadconstrain),annot=annot_loadconstrain,fmt='',xticklabels=proc_list_sorted,yticklabels=sep_list_sorted,cmap= 'viridis', vmin=0, vmax=17,cbar_kws={'label': 'ratio of extrusion time and constrain time'},ax=axs[0])
axs[0].set_ylabel('separation (kb)')
axs[0].set_xlabel('processivity (kb)')
axs[0].set_facecolor("0.8584083044982699")
axs[0].tick_params(direction='out', length=6, width=2)

                                       
fig.tight_layout(pad=1.5)
plt.savefig('Figures/'+'RelativeTimescales_heatmap.pdf',format='pdf')
plt.show()